In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from scipy import stats
import statistics
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_error, make_scorer
import seaborn as sns
import os
from scipy.stats import skew, probplot, norm
import sklearn.model_selection as ms
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV, LassoCV, Lasso, ElasticNet 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold 
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor

function to visualize the frequency of each unique value in each feature

In [2]:
#function to summarize feature
def summarize_feature(dataframe,feature):
    feature_summary = {'values':dataframe[feature].value_counts().index.tolist(),
                     'counts':dataframe[feature].value_counts().values.tolist(),
                     'frequency': dataframe[feature].value_counts(normalize = True).values.tolist()}
    feature_summary = pd.DataFrame(feature_summary)
    return feature_summary


# XGBOOST modeling

### importing data

In [3]:
train_df = pd.read_csv('./standardized splits/30/unique_train_cleaned_30.csv')
test_df = pd.read_csv('./standardized splits/30/unique_test_cleaned_30.csv')

print(f'whole_train_cleaned dataset has {train_df.shape}')
print(f'whole_test_cleaned dataset has {test_df.shape}')


whole_train_cleaned dataset has (52825, 94)
whole_test_cleaned dataset has (25902, 94)


In [5]:
#dropping the columns encounter_id and patient_nbr
train_df = train_df.drop(['encounter_id','patient_nbr'], axis = 1)
test_df = test_df.drop(['encounter_id','patient_nbr'], axis = 1)


In [6]:
train_df.sample(5)

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted,...,insulin_Up,glyburide-metformin_No,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_Steady,metformin-pioglitazone_Steady,change_No,diabetesMed_Yes
40710,1.834779,0.545888,-0.607251,-0.783594,-0.368051,-0.291878,-0.2116,-0.500531,0.823365,0,...,0,1,0,0,0,0,0,0,1,1
45411,-0.673794,-0.464698,0.973579,-0.783594,0.373126,-0.291878,-0.2116,-0.500531,0.823365,0,...,0,1,0,0,0,0,0,0,0,1
23967,0.580493,0.209026,0.310650,-0.196348,2.349597,-0.291878,-0.2116,-0.500531,0.823365,1,...,0,1,0,0,0,0,0,0,1,1
51368,0.580493,-0.127836,1.075568,-0.783594,0.496655,-0.291878,-0.2116,-0.500531,0.823365,0,...,0,1,0,0,0,0,0,0,1,0
12876,-0.046651,1.556474,0.922585,0.390898,0.126067,1.288826,-0.2116,-0.500531,0.308175,1,...,0,1,0,0,0,0,0,0,0,1


In [10]:
summarize_feature(train_df, 'readmitted')

,values,counts,frequency
0,0,48516,0.918429
1,1,4309,0.081571


In [11]:
summarize_feature(test_df, 'readmitted')

,values,counts,frequency
0,0,23419,0.904139
1,1,2483,0.095861


In [14]:
#set up the train data set into X and y
X = train_df.loc[:, train_df.columns !='readmitted']
y = train_df[['readmitted']]


In [15]:
#set up the test data into X_test and y_test
X_test = test_df.loc[:, test_df.columns !='readmitted']
y_test = test_df[['readmitted']]


In [16]:
# Apply smote technique to balance the train data
from imblearn.over_sampling import SMOTE

from collections import Counter
print('Original dataset shape {}'.format(Counter(y)))
sm = SMOTE(random_state=20)
X_new, y_new = sm.fit_sample(X, y)
print('New dataset shape {}'.format(Counter(y_new)))

Original dataset shape Counter({'readmitted': 1})
New dataset shape Counter({0: 48516, 1: 48516})


In [18]:
#Make X_new back into a df instead of a numpy array
X_new = pd.DataFrame(X_new, columns = list(X.columns))

In [20]:
# split train data set in train and valid sets for model training
X_train, X_valid, y_train, y_valid = train_test_split(X_new, y_new, test_size=0.20, random_state=42)


In [21]:
import xgboost as xgb


In [22]:
xgbc = xgb.XGBClassifier()

In [23]:
#fit X_train, y_train
xgbc.fit(X_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [28]:
# from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
# def calc_specificity(y_actual, y_pred, thresh):
#     # calculates specificity
#     return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

# def print_report(y_actual, y_pred, thresh):
    
#     auc = roc_auc_score(y_actual, y_pred)
#     accuracy = accuracy_score(y_actual, (y_pred > thresh))
#     recall = recall_score(y_actual, (y_pred > thresh))
#     precision = precision_score(y_actual, (y_pred > thresh))
#     specificity = calc_specificity(y_actual, y_pred, thresh)
#     print('AUC:%.3f'%auc)
#     print('accuracy:%.3f'%accuracy)
#     print('recall:%.3f'%recall)
#     print('precision:%.3f'%precision)
#     print('specificity:%.3f'%specificity)
#     print('prevalence:%.3f'%calc_prevalence(y_actual))
#     print(' ')
#     return auc, accuracy, recall, precision, specificity

# def calc_prevalence(y_actual):
#     return (sum(y_actual)/len(y_actual))

In [47]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix

def print_report(y_actual, y_pred, y_pred_proba):
    tp, fp, fn, tn = confusion_matrix(y_actual, y_pred).ravel()
    fpr = fp/(fp+tn)
    auc = roc_auc_score(y_actual, y_pred_proba)
    print('AUC:%.3f'%auc)
    print(f'False positive rate is {fpr}')
    print(' ')
    
#     return auc, fpr


In [57]:
thresh=0.5
y_train_pred_proba = xgbc.predict_proba(X_train)[:,1]
y_valid_pred_proba = xgbc.predict_proba(X_valid)[:,1]
y_test_pred_proba = xgbc.predict_proba(X_test)[:,1]

y_train_pred = xgbc.predict(X_train)
y_valid_pred = xgbc.predict(X_valid)
y_test_pred = xgbc.predict(X_test)

print('XGBoost')
print('Training:')
print_report(y_train, y_train_pred, y_train_pred_proba)
print('Validation:')
print_report(y_valid, y_valid_pred, y_valid_pred_proba)
print('Test:')
print_report(y_test, y_test_pred, y_test_pred_proba)


XGBoost
Training:
AUC:0.968
False positive rate is 0.005890846087207624
 
Validation:
AUC:0.968
False positive rate is 0.008269275564873049
 
Test:
AUC:0.633
False positive rate is 0.815668202764977
 


In [93]:
sorted(zip(X_train.columns,xgbc.feature_importances_), key= lambda t:t[1], reverse=True)


[('gender_Male', 0.086914375),
 ('A1Cresult_None', 0.07863705),
 ('diag_1_neoplasms', 0.06834941),
 ('discharge_disposition_id_home', 0.06400882),
 ('diag_1_respiratory', 0.052130267),
 ('race_Caucasian', 0.05046313),
 ('diag_1_digestive', 0.04399486),
 ('age', 0.043494612),
 ('time_in_hospital', 0.042303458),
 ('admission_type_id_urgent', 0.041324027),
 ('number_inpatient', 0.034621585),
 ('num_procedures', 0.034435455),
 ('diag_1_other', 0.03384887),
 ('diag_1_genitourinary', 0.030622428),
 ('discharge_disposition_id_na', 0.023144718),
 ('number_outpatient', 0.023034666),
 ('number_diagnoses', 0.022493578),
 ('insulin_No', 0.019943252),
 ('A1Cresult_>8', 0.016597651),
 ('insulin_Steady', 0.01633881),
 ('pioglitazone_No', 0.016055627),
 ('admission_source_id_urgent', 0.0143523095),
 ('diag_1_injury', 0.0140235415),
 ('diag_1_diabetes', 0.0136692785),
 ('discharge_disposition_id_hospital', 0.013433454),
 ('metformin_No', 0.012909271),
 ('diag_1_musculoskeletal', 0.012370239),
 ('number

## parameter tuning

In [58]:
xgbc1 = xgb.XGBClassifier()
xgbc1.set_params(learning_rate=0.1,subsample=0.8,n_estimators=3000,max_depth=6,
                colsample_bytree=1,objective='binary:logistic')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=3000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [59]:
#fit X_train, y_train
xgbc1.fit(X_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=3000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [62]:
thresh=0.5
y_train_pred_proba = xgbc1.predict_proba(X_train)[:,1]
y_valid_pred_proba = xgbc1.predict_proba(X_valid)[:,1]
y_test_pred_proba = xgbc1.predict_proba(X_test)[:,1]

y_train_pred = xgbc1.predict(X_train)
y_valid_pred = xgbc1.predict(X_valid)
y_test_pred = xgbc1.predict(X_test)

print('Tuned XGBoost')
print('Training:')
print_report(y_train, y_train_pred, y_train_pred_proba)
print('Validation:')
print_report(y_valid, y_valid_pred, y_valid_pred_proba)
print('Test:')
print_report(y_test, y_test_pred, y_test_pred_proba)


Tuned XGBoost
Training:
AUC:1.000
False positive rate is 2.5746652935118435e-05
 
Validation:
AUC:0.967
False positive rate is 0.015572484875644185
 
Test:
AUC:0.595
False positive rate is 0.7535885167464115
 


In [63]:
sorted(zip(X_train.columns,xgbc1.feature_importances_), key= lambda t:t[1], reverse=True)


[('discharge_disposition_id_home', 0.06871399),
 ('number_inpatient', 0.06618908),
 ('race_Caucasian', 0.04988853),
 ('A1Cresult_None', 0.039659776),
 ('gender_Male', 0.036024526),
 ('diag_1_respiratory', 0.033392355),
 ('discharge_disposition_id_hospital', 0.032423627),
 ('age', 0.031520486),
 ('discharge_disposition_id_na', 0.029182794),
 ('diag_1_other', 0.024885815),
 ('diag_1_digestive', 0.02406854),
 ('diabetesMed_Yes', 0.023830026),
 ('diag_1_neoplasms', 0.023428027),
 ('time_in_hospital', 0.019936709),
 ('diag_1_genitourinary', 0.01919183),
 ('diag_1_injury', 0.018561646),
 ('number_diagnoses', 0.016321665),
 ('num_procedures', 0.016280621),
 ('glimepiride_Steady', 0.01597332),
 ('admission_type_id_urgent', 0.015757978),
 ('pioglitazone_Steady', 0.015702289),
 ('diag_1_diabetes', 0.014206566),
 ('diag_1_musculoskeletal', 0.01376666),
 ('admission_source_id_transfer', 0.012640475),
 ('admission_source_id_urgent', 0.011597336),
 ('rosiglitazone_Up', 0.011341949),
 ('metformin_No'